# We start with 2 data sources.
## The first is the file with the full name in the format of 2nd,tld,3rd
## The second is the file with existing domain names 


In [1]:
# Set up our file locations here
fulldomainfile = '/mnt/e/Rapid7/FDNSV2/2020-09-25-1600992617-fdns_any_domains_with_tld'
domainnamefile = '/mnt/d/flat_all'
workingdirectory = '/mnt/d/thirdlevelworking/'

In [ ]:
##THIS IS RUNITFASTER.PY

#lets try it with the database this time....
import io
import subprocess
import pyodbc
import tldextract as tlde
import csv
import sys

#cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
#sql = "select * from orbisData"
#cursor = cnxn.cursor()
#cursor.execute(sql)

#NOTE:  I HAVE THIS DISABLED IN THE JUPYTER NOTEBOOK BECAUSE IT CAN TAKE A LONG TIME TO RUN....
#       It works, it just is probably better to run in a terminal to keep an eye on it.
#
#  comment out the following line to run it for realz
#assert(False)

#infile = sys.argv[1] #'/mnt/e/Rapid7/FDNSV2/firstlevel_sorted.csv'
infile = '/mnt/d/secondlevelworking/justcomsorted.csv'
outfile = '/mnt/d/secondlevelworking/counted.csv'

print("Reading ", infile, " as the input file")

limitCount = 10000
limitCurrentCount = 0
totalcount = 0

current2ndLevel = ''
last2ndLevel = ''
current2ndLevelCount = 0
#print('hello')
linecount = 0

fresh = False
bigChongus = {}


#this is doing it this way because 
# 1) separate disc operations (all read, and then all write, using memory as the intermediary)
# 2) Limit the memory usage.  When I ran it all together, it did not fit in memory, and it crashes
#    A large portion of the names have only one or two entries, so we will do those separately.
with open(outfile, 'w') as f:
    #---------------------------------------------------------
    #--- First run through, only names with 1 entry
    #---------------------------------------------------------
    with open(infile) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')

        for row in readCSV:
            linecount = linecount + 1
            if row[0] != last2ndLevel:
                if current2ndLevelCount != 0:
                    if current2ndLevelCount == 1:
                        bigChongus[last2ndLevel] = current2ndLevelCount
                        totalcount = totalcount + current2ndLevelCount
                    #f.write(last2ndLevel + "," + str(current2ndLevelCount) + "\n");

                    fresh = True

                current2ndLevelCount = 1
                last2ndLevel = row[0]
            else:
                current2ndLevelCount = current2ndLevelCount + 1
                fresh = False

            if linecount % 10000000 == 0:
                print("proccessing at line: ", f'{linecount:,}', flush=True)

            limitCurrentCount = limitCurrentCount + 1
            
    if fresh == False:
        if current2ndLevelCount == 1:
            bigChongus[last2ndLevel] = current2ndLevelCount
            totalcount = totalcount + current2ndLevelCount
    
    fresh = True
        
    for key in bigChongus:
        f.write(key + "," + str(bigChongus[key]) + "\n")        
    bigChongus.clear()
       
    #---------------------------------------------------------
    #--- Second run through, only names with 2 entry
    #---------------------------------------------------------
    with open(infile) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')

        for row in readCSV:
            linecount = linecount + 1
            if row[0] != last2ndLevel:
                if current2ndLevelCount != 0:
                    if current2ndLevelCount == 2:
                        bigChongus[last2ndLevel] = current2ndLevelCount
                        totalcount = totalcount + current2ndLevelCount

                    fresh = True

                current2ndLevelCount = 1
                last2ndLevel = row[0]
            else:
                current2ndLevelCount = current2ndLevelCount + 1
                fresh = False

            if linecount % 10000000 == 0:
                print("proccessing at line: ", f'{linecount:,}', flush=True)

            limitCurrentCount = limitCurrentCount + 1
    
    if fresh == False:
        if current2ndLevelCount == 2:
            bigChongus[last2ndLevel] = current2ndLevelCount
            totalcount = totalcount + current2ndLevelCount
    
    fresh = True
    
    for key in bigChongus:
        f.write(key + "," + str(bigChongus[key]) + "\n")        
    bigChongus.clear()
       
    linecount = 0
    #---------------------------------------------------------
    #--- Third run through, only names with >2 entries
    #---------------------------------------------------------
    with open(infile) as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')

        for row in readCSV:
            linecount = linecount + 1
            if row[0] != last2ndLevel:
                if current2ndLevelCount != 0:
                    if current2ndLevelCount > 2:
                        bigChongus[last2ndLevel] = current2ndLevelCount
                        totalcount = totalcount + current2ndLevelCount
                    
                    fresh = True

                current2ndLevelCount = 1
                last2ndLevel = row[0]
            else:
                current2ndLevelCount = current2ndLevelCount + 1
                fresh = False

            if linecount % 10000000 == 0:
                print("proccessing at line: ", f'{linecount:,}', flush=True)

            limitCurrentCount = limitCurrentCount + 1
    
    if fresh == False:
        if current2ndLevelCount > 2:
            bigChongus[last2ndLevel] = current2ndLevelCount
            totalcount = totalcount + current2ndLevelCount
    
    fresh = True
    
    for key in bigChongus:
        f.write(key + "," + str(bigChongus[key]) + "\n")        
    bigChongus.clear()
    
    #---------------------------------------------------------
    #--- Done with run throughs
    #---------------------------------------------------------

print("Sanity check.  These should match")
print("linecount: ", linecount)
print("totalcount: ", totalcount)
assert(linecount == totalcount)


Reading  /mnt/d/secondlevelworking/justcomsorted.csv  as the input file
proccessing at line:  10,000,000
proccessing at line:  20,000,000
proccessing at line:  30,000,000
proccessing at line:  40,000,000
proccessing at line:  50,000,000
proccessing at line:  60,000,000
proccessing at line:  70,000,000


In [3]:

import io
import subprocess
import pyodbc
import tldextract as tlde
import csv
import sys

sourceFile = 'e:\Rapid7\FDNSV2\split\counted.csv'

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
#sql = "select * from orbisData"
cursor = cnxn.cursor()
#cursor.execute(sql)

#create temparary table
temptablesql = "select secondlevelname, count into #tempCounter from secondlevelcounterfaster where 0 = 1;"
cnxn.cursor().execute(temptablesql)

indexSql = "create unique nonclustered index [IX_tempCounter] on #tempCounter([secondlevelname] asc) with (PAD_INDEX = OFF, STATISTICS_NORECOMPUTE = OFF, SORT_IN_TEMPDB = OFF, IGNORE_DUP_KEY = OFF, DROP_EXISTING = OFF, ONLINE = OFF, ALLOW_ROW_LOCKS = ON, ALLOW_PAGE_LOCKS = ON) ON [PRIMARY]"
cursor.execute(indexSql)

#bulk insert into temp table
bulkInsertSql = "bulk insert #tempCounter from '" + sourceFile + "' with (ROWTERMINATOR = '0x0a', FORMAT='CSV')"
print("Running: ", bulkInsertSql)

cursor.execute(bulkInsertSql)
cursor.commit()

testselect = "select count(*) from #tempCounter"
cursor.execute(testselect)

while True:
    row = cursor.fetchone()
    if not row:
        break;
    print(row)
    
#do the update to the real table here
bigUpdateSql = "update t2 set t2.count = t1.count from secondlevelcounterfaster t2 join #tempCounter t1 on t1.secondlevelname = t2.secondlevelname"
cursor.execute(bigUpdateSql)
cursor.commit()

dropTempTableSql = "drop table #tempCounter"
cursor.execute(dropTempTableSql)

print("done")

Running:  bulk insert #tempCounter from 'e:\Rapid7\FDNSV2\split\counted.csv' with (ROWTERMINATOR = '0x0a', FORMAT='CSV')
(48049738, )
done


In [2]:
##THIS IS SMARTERGREPITFASTER.PY

#lets try it with the database this time....
import io
import subprocess
import pyodbc
import tldextract as tlde
import csv
import sys

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
cnxn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
#sql = "select * from orbisData"
cursor = cnxn.cursor()
cursor2 = cnxn2.cursor()
#cursor.execute(sql)

#NOTE:  I HAVE THIS DISABLED IN THE JUPYTER NOTEBOOK BECAUSE IT CAN TAKE A LONG TIME TO RUN....
#       It works, it just is probably better to run in a terminal to keep an eye on it.
#
#  comment out the following line to run it for realz
#assert(False)

secondlevelnameselect = "select secondlevelname from secondlevelcounterfaster where smartergrepcount = 0 and count >= 0 and smartergrepped = 0 order by count desc"
cursor.execute(secondlevelnameselect)

while True:
    row = cursor.fetchone()
    if not row:
        break
    grepstring = row[0]
    pipe = subprocess.Popen("LC_ALL=C grep  -c \"" + grepstring.lower() + "\.com\" /mnt/r/flat_all", shell=True, stdout=subprocess.PIPE)

    count = int(pipe.communicate()[0])
    print("count of " + grepstring + " is: ", str(count))

    cursor2.execute("update secondlevelcounterfaster set smartergrepped = 1, smartergrepcount = " + str(count) + " where secondlevelname = '" + grepstring + "'")
    
    cnxn2.commit()

cnxn.close()
cnxn2.close()

print("done")

count of demon is:  3624
count of kaojiwang is:  3
count of luhongfu is:  1
count of patch is:  25737
count of buycheaptoy is:  1
count of vb955 is:  2
count of elleny is:  152
count of syjtw is:  7
count of taclgcjx is:  1
count of bbobn is:  1
count of love465 is:  1
count of fenlei001 is:  1
count of aaa99999 is:  2
count of 15wh is:  15
count of smartgslb is:  1
count of e7998 is:  6
count of hotels4trip is:  1
count of gleeze is:  6
count of yundunwaf is:  2
count of donet is:  989
count of alternativagratis is:  1
count of monmouth is:  716
count of knight-electrical is:  2
count of pizzamaniagame is:  1
count of lflink is:  154
count of reliablehosting is:  37
count of jiuyangjidian is:  1
count of hfxsky is:  1
count of 13814884863 is:  1
count of 03card is:  8
count of reviewsmanagement is:  22
count of szkopod is:  1
count of hlqcrc is:  1
count of 921922 is:  10
count of p6is is:  2
count of floristforall is:  1
count of qq293yy is:  1
count of zjdwdq is:  1
count of serveus

count of dumb1 is:  4
count of igodigital is:  60
count of lylsjd is:  1
count of yulesk is:  1
count of uggdaily is:  2
count of campbrainregistration is:  2
count of bizcn is:  47
count of tuhrss is:  1
count of jotfom is:  1
count of ikidane is:  4
count of bestpainfix is:  1
count of edukoo is:  3
count of diysoez is:  1
count of ikwb is:  15
count of mingyuanjidian is:  1
count of nevadabesthotels is:  1
count of space163 is:  1
count of 178touzi is:  1
count of bitacoras is:  30
count of 258oo is:  5
count of usaht is:  10
count of aweber is:  716
count of designextreme is:  8
count of leadsboy is:  1
count of agdpr is:  51
count of groupsite is:  434
count of topexecutivecareers is:  1
count of shyibang is:  1
count of 007idea is:  1
count of gwbiv is:  1
count of hotelinsamui is:  1
count of designsoez is:  1
count of ibackpage is:  27
count of ecommtools is:  2
count of descargasbajar is:  1
count of boostersite is:  30
count of 011ok is:  6
count of moicha is:  1
count of lif

count of careerscreative is:  1
count of kam-sun is:  1
count of com-girlfriends is:  1
count of 089y is:  16
count of quickvmail is:  1
count of hotel-in-pattaya is:  2
count of gog is:  2804
count of 18avp is:  1
count of ospreyusa is:  2
count of hotel-in-cancun is:  2
count of ccsw012 is:  1
count of xyjszl is:  1
count of 982004 is:  15
count of hzmiwei is:  1
count of h5628 is:  8
count of ticketbud is:  8
count of forumpolish is:  1
count of tuding123 is:  1
count of turn-thegame is:  1
count of ekd7 is:  13
count of mittum is:  1
count of b8a7 is:  2
count of shopgate is:  39
count of youhoser is:  3
count of betterlinuxtesting is:  1
count of dofreedownload is:  1
count of eomoo is:  15
count of 8971928 is:  1
count of sahibinden is:  729
count of directnets is:  2
count of ipodnanoflaw is:  1
count of lolclassifieds is:  1
count of czyzpp is:  1
count of windows8compatible is:  1
count of dqjrxl is:  1
count of 6dli is:  2
count of mapyourshow is:  2
count of ziprf is:  1
cou

count of comendo is:  74
count of area-porting is:  0
count of hdjwg is:  1
count of kyusyu-onsen is:  1
count of cstudio is:  10621
count of kontent is:  233
count of sssbw is:  6
count of mercedes-benz is:  518
count of doorna is:  6
count of criticalimpactinc is:  1
count of savingcall is:  9
count of qvt8 is:  3
count of photodeck is:  3
count of tiaplex is:  4
count of enligne-uk is:  1
count of fsgdybxg is:  1
count of hduhgs is:  1
count of cyber-world is:  7
count of 160by2mail is:  1
count of gztaiyufz is:  1
count of dominiotemporarioidc is:  1
count of caogenfun is:  1
count of dgshq is:  2
count of rogershsa is:  1
count of siemv is:  1
count of inwitelecom is:  1
count of 683721 is:  7
count of hightechautoaccessory is:  1
count of acs-pitt is:  1
count of hd-jl is:  1
count of nowuru is:  2
count of home-security-leader is:  1
count of yu-nagi is:  2
count of stagingdm is:  2
count of meydansozluk is:  1
count of tzmingye is:  1
count of 604868 is:  1
count of santiagotop

count of 24288946 is:  1
count of etlong is:  237
count of dushifang is:  1
count of chatovod is:  10
count of realresultfitness is:  1
count of recruiterbox is:  8
count of luo-gang is:  1
count of auak is:  69
count of gbs2u is:  3
count of pershastudia is:  1
count of n1987 is:  369
count of ballsgold is:  2
count of yunduncdn is:  2
count of fargrid is:  1
count of 86mai is:  10
count of si999 is:  68
count of cysqxj is:  1
count of ticketsclarkston is:  1
count of 136zg is:  1
count of virserum is:  4
count of lyxwjyz is:  1
count of ispvds is:  1
count of cdcrx is:  5
count of 37387118 is:  1
count of 789339 is:  11
count of bunsga is:  1
count of hoteis-noreste-de-brasil is:  1
count of asiawhere is:  1
count of pectry is:  4
count of carestack is:  5
count of fqf7 is:  6
count of hotelsofmenorca is:  1
count of qdkfq148 is:  1
count of mxvirtual is:  5
count of minyanqian is:  1
count of greektowncasino is:  6
count of shopingfa is:  1
count of zhangpuxuan is:  1
count of meill

OSError: [Errno 12] Cannot allocate memory

In [4]:
##THIS IS GREPITFASTER.PY

#lets try it with the database this time....
import io
import subprocess
import pyodbc
import tldextract as tlde
import csv
import sys

cnxn = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
cnxn2 = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server};SERVER=localhost;DATABASE=DomainResearch;UID=dr;PWD=dr2020')
#sql = "select * from orbisData"
cursor = cnxn.cursor()
cursor2 = cnxn2.cursor()
#cursor.execute(sql)

#NOTE:  I HAVE THIS DISABLED IN THE JUPYTER NOTEBOOK BECAUSE IT CAN TAKE A LONG TIME TO RUN....
#       It works, it just is probably better to run in a terminal to keep an eye on it.
#
#  comment out the following line to run it for realz
#assert(False)

secondlevelnameselect = "select secondlevelname from secondlevelcounterfaster where dumbgrepcount = 0 and count >= 0 and grepped = 0 order by count desc"
cursor.execute(secondlevelnameselect)

while True:
    row = cursor.fetchone()
    if not row:
        break
    grepstring = row[0]
    pipe = subprocess.Popen("LC_ALL=C grep -c \"" + grepstring.lower() + "\" /mnt/d/flat_all LC_ALL=C ", shell=True, stdout=subprocess.PIPE)

    count = int(pipe.communicate()[0])
    print("count of " + grepstring + " is: ", str(count))

    cursor2.execute("update secondlevelcounterfaster set grepped = 1, dumbgrepcount = " + str(count) + " where secondlevelname = '" + grepstring + "'")
    
    cnxn2.commit()

cnxn.close()
cnxn2.close()

print("done")
assert(False)

count of vnsfind is:  0
count of bphost24 is:  0
count of vnsgold is:  0
count of vnskid is:  0
count of lc-cn-n1-thovg is:  0
count of cloud116411 is:  0
count of vnsfax is:  0
count of bphost20 is:  0
count of vnscool is:  0
count of auditionways is:  0
count of vnswine is:  0
count of vnsgas is:  0
count of bphost21 is:  0
count of vnsspain is:  0
count of vnsautos is:  0
count of vnsbiz is:  0
count of vnsski is:  0
count of vnspda is:  0
count of cloud117411 is:  0
count of bphost22 is:  0
count of maklimp402587 is:  0
count of auopeninghoursau is:  0


SystemError: <class 'pyodbc.Error'> returned a result with an error set